In [4]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [5]:
from pdf2image import convert_from_path
import pytesseract
pdfs = "./Articles/DataForLife.pdf"
pages = convert_from_path(pdfs, 350)
# print(pages)

i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"  
    page.save(image_name, "JPEG")
    i = i+1 

In [6]:

# use this command to install open cv2
# pip install opencv-python

# use this command to install PIL
# pip install Pillow

import cv2
from PIL import Image

def mark_region(image_path):
    print(image_path)
    im = cv2.imread(image_path)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_coordinates = []
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)

        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])

        if y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])


    return image, line_items_coordinates

In [1]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

img = cv2.imread('Page_1.jpg')

custom_config = r'--psm 3'
# data = pytesseract.image_to_string(img, config=custom_config)
# print(data)
# df = pd.DataFrame(data = data)
# display(df)
# df.head(10)
d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
print(d.keys())
df = pd.DataFrame(d)
df.to_csv('./trialData.csv')
display(df)

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,2398,3404,-1,
1,2,1,1,0,0,0,306,345,469,66,-1,
2,3,1,1,1,0,0,306,345,469,66,-1,
3,4,1,1,1,1,0,306,345,469,66,-1,
4,5,1,1,1,1,1,306,345,245,66,96,Original
...,...,...,...,...,...,...,...,...,...,...,...,...
459,5,1,10,1,1,9,1596,3228,39,29,92,"11,"
460,5,1,10,1,1,10,1648,3228,22,29,92,"3,"
461,5,1,10,1,1,11,1683,3228,122,24,91,317-333
462,4,1,10,1,2,0,1017,3265,389,33,-1,


In [7]:
#Grouping the text by their block numbers
processedDf = df.groupby(
   ['block_num']
).agg(
    {
         'left':min,  
         'top': min,  
         'width': max,
         'height': max,
         'text': (lambda x: ' '.join(x))
    }
)

grouped_df = processedDf.reset_index()
grouped_df.to_csv('./paraBlocksTrial.csv')

display(grouped_df)
# Creating a visualization that showcases the boxes 
for i in range(len(grouped_df)):
    # print(grouped_df.loc[i,'left'])
#     if int(d['conf'][i]) > 60:
    (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


cv2.imshow('img', img)
cv2.waitKey(5000)

# n_boxes = len(d['text'])
# print(n_boxes)
# for i in range(n_boxes):
#     if int(d['conf'][i]) > 60:
#         (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
#         img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


# cv2.imshow('img', img)
# cv2.waitKey(5000)


,block_num,left,top,width,height,text
0,0,0,0,2398,3404,
1,1,306,345,469,66,Original Article
2,2,304,457,1745,211,Data for life: Wearable technology and the...
3,3,306,769,353,30,Natasha Dow Schill
4,4,306,831,1692,88,"Department of Media, Culture and Communicat..."
5,5,304,1063,1817,804,"Abstract Over the last 5 years, wearable te..."
6,6,306,1955,1465,43,Keywords: digital health; wearable technolo...
7,7,304,2307,1817,676,"In January, 2014, exactly 1 week after Pres..."
8,8,328,3075,1793,88,1 The seed for this article was a short pie...
9,9,302,3186,1821,6,


-1

In [4]:
import cv2
import pytesseract

filename = 'Page_1.jpg'

# read the image and get the dimensions
img = cv2.imread(filename)
h, w, _ = img.shape # assumes color image
# run tesseract, returning the bounding boxes
boxes = pytesseract.image_to_boxes(img) # also include any config options you use

# draw the bounding boxes on the image
for b in boxes.splitlines():
    b = b.split(' ')
    img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)

# show annotated image and wait for keypress
cv2.imshow(filename, img)
cv2.waitKey(0)

111

In [2]:
import cv2
import numpy as np

# Load image, grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('Page_1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Create rectangular structuring element and dilate
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.dilate(thresh, kernel, iterations=4)

# Find contours and draw rectangle
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)

cv2.imshow('thresh', thresh)
cv2.imshow('dilate', dilate)
# cv2.imshow('image', image)
cv2.waitKey(5000)

-1